In [5]:
import streamlit as st
from PyPDF2 import PdfReader
#After reading the pdf, convert it to vectors
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as generativeai
from langchain.vectorstores import FAISS
from langchain_google_genai import  ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv


In [ ]:

load_dotenv()

generativeai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        read_pdf = PdfReader(pdf)
        for page in read_pdf.pages:
            text+=page.extract_text
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chuncks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
    
def get_conversational_chain():
    prompt_template = """
        Answer the quetion as detailed from the provided context, make usre to provide all the details, if the answer is not in
        the provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
        Context:\n {context}?\n
        Question: \n {question}\n
        
        Answer: 
    """
    model = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.3)
    
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type='stuff', prompt=prompt)
    return chain

def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    
    new_db = FAISS.load_local("faiss_index", embeddings )
    docs = new_db.similarity_search(user_question)
    
    chain = get_conversational_chain()
    
    reponse = chain(
        {"input_documents": docs,
         "question": user_question
         
        },
        return_only_outputs=True
    )
    print(response) 
    st.write("Reply: ", response["output_text"])
    
    
def main():
    st.set_page_config("PDF AI")
    st.header("Chat with me")
    
    user_question = st.text_input("Ask a question")
    
    if user_question:
        user_input(user_question)
        
    with st.sidebar:
        st.title("Menu: ")
        pdf_docs = st.file_uploader("Upload your pdf files")
        if st.button("Submit"):
            with st.spinner("Processing..."):
                raw_text = get_pdf_text(pdf_docs)
                text_chunks = get_text_chunks(raw_text)
                get_vector_store(text_chunks)
                st.success("Done")
            

if __name__ == "__main__":
    main()
            
        
        